In [1]:
import numpy as np
import pandas as pd
import os

from shl.prepare import fill_na, calculate_abs_values, calculate_change, calculate_pct_change, calculate_window, calculate_shift, fillna_agg_by_label
from typing import List, Optional, Tuple

data_path = '../data'

pd.set_option('display.max_columns', None)

In [2]:
def prepare_features(df, df_label, window_sizes, window_center, window_functions, shift_periods, shift_column_patterns, fill_limit):  
        
    df = df.copy()    
   
    for col in df.columns:
        
        if col == 'epoch_time':
            df[col] = df[col].astype('int64')        
        
        elif df[col].dtype != 'object':
            df[col] = df[col].astype('float32')    
    
   
    
    # Add time rounded to seconds
    df['epoch_time'] = df['epoch_time'].round(-3)
        
    # Save original timestamp before it gets filled
    df_time = df[['epoch_time']].drop_duplicates()
  
    # Group values    
    df = df[['epoch_time', 'cell_type', 'asuLevel']].groupby(['epoch_time', 'cell_type'])['asuLevel'].median().reset_index(name='asuLevel') 
    
    # Move cell types to columns
    df = df.pivot(index='epoch_time', columns='cell_type', values='asuLevel') \
           .rename(columns={'LTE': 'asuLevel_LTE', 'GSM': 'asuLevel_GSM', 'WCDMA': 'WCDMA_asuLevel'}) \
           .reset_index()       
     
    # Join with label
    # (use the same df if label df in None)    
    if df_label is None:
        df_label = df[['epoch_time']]       
        
    df = df_label.merge(df, on='epoch_time', how='outer').drop('label', axis=1, errors='ignore')       
    
    # Ensure that values are sorted before calculating diff
    df = df.sort_values(by='epoch_time')        
        
    # Calculate percent change
    df_pct = calculate_pct_change(df)    
    df_pct.drop(['epoch_time_pct_change'], axis=1, inplace=True) 
    fill_na(df_pct, fill_limit)
    
    # Merge new features back to the main dataframe
    df = df[['epoch_time']].merge(df_pct, left_index=True, right_index=True, how='left')       
    
    # Add moving windows features
    df = calculate_window(df, columns=None, functions=window_functions, window_sizes=window_sizes, window_center=True, fill_limit=fill_limit)       
    
    # Add shifted features
    calculate_shift(df, periods=shift_periods, columns_patterns=shift_column_patterns, fill_limit=fill_limit)             

    df = df.merge(df_time)
   
    return fillna_agg_by_label(df, df_label, fill_limit, 'epoch_time')

In [5]:
# Run processing

settings = {
'window_sizes': [5, 10, 30, 60, 300],
'window_center': True,
'window_functions': ['mean', 'std'],
'shift_periods': [5, 10, 30, 60, 300],
'shift_column_patterns': ['window_5_', 'window_10_', 'window_30_', 'window_60_', 'window_300_'],
'fill_limit': 30,
}

join_label = True

input_file_name = 'Cells.parquet'
input_file_name_label = 'Label.parquet'
input_file_name_label_test = 'Label_idx.parquet'
data_types = ['validate', 'train', 'test']
# data_types = ['test']
output_file_name = 'features_' + str.lower(input_file_name.split('.')[0]) + '.parquet'


for data_type in data_types:
    file_in = os.path.join(data_path, data_type, input_file_name)
    df = pd.read_parquet(file_in)
    df_label = None
    file_in_label = os.path.join(data_path, data_type, input_file_name_label_test if data_type == 'test' else input_file_name_label)
    if join_label is True and os.path.exists(file_in_label):
        df_label = pd.read_parquet(file_in_label)
    df = prepare_features(df, df_label, **settings)
    file_out = os.path.join(data_path, data_type, output_file_name)
    df.to_parquet(file_out, index=False)
   


In [35]:
df

,epoch_time,asuLevel_GSM_pct_change,asuLevel_LTE_pct_change,WCDMA_asuLevel_pct_change,asuLevel_GSM_pct_change_window_5_mean,asuLevel_GSM_pct_change_window_5_std,asuLevel_LTE_pct_change_window_5_mean,asuLevel_LTE_pct_change_window_5_std,WCDMA_asuLevel_pct_change_window_5_mean,WCDMA_asuLevel_pct_change_window_5_std,asuLevel_GSM_pct_change_window_10_mean,asuLevel_GSM_pct_change_window_10_std,asuLevel_LTE_pct_change_window_10_mean,asuLevel_LTE_pct_change_window_10_std,WCDMA_asuLevel_pct_change_window_10_mean,WCDMA_asuLevel_pct_change_window_10_std,asuLevel_GSM_pct_change_window_30_mean,asuLevel_GSM_pct_change_window_30_std,asuLevel_LTE_pct_change_window_30_mean,asuLevel_LTE_pct_change_window_30_std,WCDMA_asuLevel_pct_change_window_30_mean,WCDMA_asuLevel_pct_change_window_30_std,asuLevel_GSM_pct_change_window_60_mean,asuLevel_GSM_pct_change_window_60_std,asuLevel_LTE_pct_change_window_60_mean,asuLevel_LTE_pct_change_window_60_std,WCDMA_asuLevel_pct_change_window_60_mean,WCDMA_asuLevel_pct_change_window_60_std,asuLevel_GSM_pct_change_window_300_mean,asuLevel_GSM_pct_change_window_300_std,asuLevel_LTE_pct_change_window_300_mean,asuLevel_LTE_pct_change_window_300_std,WCDMA_asuLevel_pct_change_window_300_mean,WCDMA_asuLevel_pct_change_window_300_std,asuLevel_GSM_pct_change_window_5_mean_shift_5_past,asuLevel_GSM_pct_change_window_5_std_shift_5_past,asuLevel_LTE_pct_change_window_5_mean_shift_5_past,asuLevel_LTE_pct_change_window_5_std_shift_5_past,WCDMA_asuLevel_pct_change_window_5_mean_shift_5_past,WCDMA_asuLevel_pct_change_window_5_std_shift_5_past,asuLevel_GSM_pct_change_window_5_mean_shift_5_future,asuLevel_GSM_pct_change_window_5_std_shift_5_future,asuLevel_LTE_pct_change_window_5_mean_shift_5_future,asuLevel_LTE_pct_change_window_5_std_shift_5_future,WCDMA_asuLevel_pct_change_window_5_mean_shift_5_future,WCDMA_asuLevel_pct_change_window_5_std_shift_5_future,asuLevel_GSM_pct_change_window_10_mean_shift_10_past,asuLevel_GSM_pct_change_window_10_std_shift_10_past,asuLevel_LTE_pct_change_window_10_mean_shift_10_past,asuLevel_LTE_pct_change_window_10_std_shift_10_past,WCDMA_asuLevel_pct_change_window_10_mean_shift_10_past,WCDMA_asuLevel_pct_change_window_10_std_shift_10_past,asuLevel_GSM_pct_change_window_10_mean_shift_10_future,asuLevel_GSM_pct_change_window_10_std_shift_10_future,asuLevel_LTE_pct_change_window_10_mean_shift_10_future,asuLevel_LTE_pct_change_window_10_std_shift_10_future,WCDMA_asuLevel_pct_change_window_10_mean_shift_10_future,WCDMA_asuLevel_pct_change_window_10_std_shift_10_future,asuLevel_GSM_pct_change_window_30_mean_shift_30_past,asuLevel_GSM_pct_change_window_30_std_shift_30_past,asuLevel_LTE_pct_change_window_30_mean_shift_30_past,asuLevel_LTE_pct_change_window_30_std_shift_30_past,WCDMA_asuLevel_pct_change_window_30_mean_shift_30_past,WCDMA_asuLevel_pct_change_window_30_std_shift_30_past,asuLevel_GSM_pct_change_window_30_mean_shift_30_future,asuLevel_GSM_pct_change_window_30_std_shift_30_future,asuLevel_LTE_pct_change_window_30_mean_shift_30_future,asuLevel_LTE_pct_change_window_30_std_shift_30_future,WCDMA_asuLevel_pct_change_window_30_mean_shift_30_future,WCDMA_asuLevel_pct_change_window_30_std_shift_30_future,asuLevel_GSM_pct_change_window_60_mean_shift_60_past,asuLevel_GSM_pct_change_window_60_std_shift_60_past,asuLevel_LTE_pct_change_window_60_mean_shift_60_past,asuLevel_LTE_pct_change_window_60_std_shift_60_past,WCDMA_asuLevel_pct_change_window_60_mean_shift_60_past,WCDMA_asuLevel_pct_change_window_60_std_shift_60_past,asuLevel_GSM_pct_change_window_60_mean_shift_60_future,asuLevel_GSM_pct_change_window_60_std_shift_60_future,asuLevel_LTE_pct_change_window_60_mean_shift_60_future,asuLevel_LTE_pct_change_window_60_std_shift_60_future,WCDMA_asuLevel_pct_change_window_60_mean_shift_60_future,WCDMA_asuLevel_pct_change_window_60_std_shift_60_future,asuLevel_GSM_pct_change_window_300_mean_shift_300_past,asuLevel_GSM_pct_change_window_300_std_shift_300_past,asuLevel_LTE_pct_change_window_300_mean_shift_300_past

In [21]:
df.isnull().sum().sum()

0

### Export for analysis

In [6]:
# Join labels for analysis
# features_file = '../data/test/features_cells.parquet'
# features = pd.read_parquet(features_file)


# cells_raw = '../data/train/Cells.parquet'
# cells_raw_df = pd.read_parquet(cells_raw)

# cells_raw_df['epoch_time_s'] = cells_raw_df['epoch_time'].round(-3)

# label_file = './data/validate/Label.parquet'
# label = pd.read_parquet(label_file)

# features = label.merge(features).sort_values(by='epoch_time')
# features.to_csv('./data/cells_analysis.csv')
# features

,epoch_time_id,epoch_time,asuLevel_GSM_pct_change,asuLevel_LTE_pct_change,WCDMA_asuLevel_pct_change,asuLevel_GSM_pct_change_window_5_mean,asuLevel_GSM_pct_change_window_5_std,asuLevel_LTE_pct_change_window_5_mean,asuLevel_LTE_pct_change_window_5_std,WCDMA_asuLevel_pct_change_window_5_mean,WCDMA_asuLevel_pct_change_window_5_std,asuLevel_GSM_pct_change_window_10_mean,asuLevel_GSM_pct_change_window_10_std,asuLevel_LTE_pct_change_window_10_mean,asuLevel_LTE_pct_change_window_10_std,WCDMA_asuLevel_pct_change_window_10_mean,WCDMA_asuLevel_pct_change_window_10_std,asuLevel_GSM_pct_change_window_30_mean,asuLevel_GSM_pct_change_window_30_std,asuLevel_LTE_pct_change_window_30_mean,asuLevel_LTE_pct_change_window_30_std,WCDMA_asuLevel_pct_change_window_30_mean,WCDMA_asuLevel_pct_change_window_30_std,asuLevel_GSM_pct_change_window_60_mean,asuLevel_GSM_pct_change_window_60_std,asuLevel_LTE_pct_change_window_60_mean,asuLevel_LTE_pct_change_window_60_std,WCDMA_asuLevel_pct_change_window_60_mean,WCDMA_asuLevel_pct_change_window_60_std,asuLevel_GSM_pct_change_window_300_mean,asuLevel_GSM_pct_change_window_300_std,asuLevel_LTE_pct_change_window_300_mean,asuLevel_LTE_pct_change_window_300_std,WCDMA_asuLevel_pct_change_window_300_mean,WCDMA_asuLevel_pct_change_window_300_std,asuLevel_GSM_pct_change_window_5_mean_shift_5_past,asuLevel_GSM_pct_change_window_5_std_shift_5_past,asuLevel_LTE_pct_change_window_5_mean_shift_5_past,asuLevel_LTE_pct_change_window_5_std_shift_5_past,WCDMA_asuLevel_pct_change_window_5_mean_shift_5_past,WCDMA_asuLevel_pct_change_window_5_std_shift_5_past,asuLevel_GSM_pct_change_window_5_mean_shift_5_future,asuLevel_GSM_pct_change_window_5_std_shift_5_future,asuLevel_LTE_pct_change_window_5_mean_shift_5_future,asuLevel_LTE_pct_change_window_5_std_shift_5_future,WCDMA_asuLevel_pct_change_window_5_mean_shift_5_future,WCDMA_asuLevel_pct_change_window_5_std_shift_5_future,asuLevel_GSM_pct_change_window_10_mean_shift_10_past,asuLevel_GSM_pct_change_window_10_std_shift_10_past,asuLevel_LTE_pct_change_window_10_mean_shift_10_past,asuLevel_LTE_pct_change_window_10_std_shift_10_past,WCDMA_asuLevel_pct_change_window_10_mean_shift_10_past,WCDMA_asuLevel_pct_change_window_10_std_shift_10_past,asuLevel_GSM_pct_change_window_10_mean_shift_10_future,asuLevel_GSM_pct_change_window_10_std_shift_10_future,asuLevel_LTE_pct_change_window_10_mean_shift_10_future,asuLevel_LTE_pct_change_window_10_std_shift_10_future,WCDMA_asuLevel_pct_change_window_10_mean_shift_10_future,WCDMA_asuLevel_pct_change_window_10_std_shift_10_future,asuLevel_GSM_pct_change_window_30_mean_shift_30_past,asuLevel_GSM_pct_change_window_30_std_shift_30_past,asuLevel_LTE_pct_change_window_30_mean_shift_30_past,asuLevel_LTE_pct_change_window_30_std_shift_30_past,WCDMA_asuLevel_pct_change_window_30_mean_shift_30_past,WCDMA_asuLevel_pct_change_window_30_std_shift_30_past,asuLevel_GSM_pct_change_window_30_mean_shift_30_future,asuLevel_GSM_pct_change_window_30_std_shift_30_future,asuLevel_LTE_pct_change_window_30_mean_shift_30_future,asuLevel_LTE_pct_change_window_30_std_shift_30_future,WCDMA_asuLevel_pct_change_window_30_mean_shift_30_future,WCDMA_asuLevel_pct_change_window_30_std_shift_30_future,asuLevel_GSM_pct_change_window_60_mean_shift_60_past,asuLevel_GSM_pct_change_window_60_std_shift_60_past,asuLevel_LTE_pct_change_window_60_mean_shift_60_past,asuLevel_LTE_pct_change_window_60_std_shift_60_past,WCDMA_asuLevel_pct_change_window_60_mean_shift_60_past,WCDMA_asuLevel_pct_change_window_60_std_shift_60_past,asuLevel_GSM_pct_change_window_60_mean_shift_60_future,asuLevel_GSM_pct_change_window_60_std_shift_60_future,asuLevel_LTE_pct_change_window_60_mean_shift_60_future,asuLevel_LTE_pct_change_window_60_std_shift_60_future,WCDMA_asuLevel_pct_change_window_60_mean_shift_60_future,WCDMA_asuLevel_pct_change_window_60_std_shift_60_future,asuLevel_GSM_pct_change_window_300_mean_shift_300_past,asuLevel_GSM_pct_change_window_300_std_shift_300_past,asuLevel_LTE_pct_change_window_300_mean_

In [12]:
# cells_raw_df.dtypes

epoch_time        int64
num_entries       int64
cell_type        object
isRegistered      int64
ci              float64
MCC               int64
MNC               int64
PCI             float64
TAC             float64
asuLevel          int64
dBm               int64
level             int64
cid             float64
lac             float64
PSC             float64
dtype: object